In [10]:
import torch
from torch import nn
import dam2s
import cooper
# from VFeatureExtractor import VFeatureExtractor
# from DFeatureExtractor import DFeatureExtractor

In [15]:
# have not got the data yet. using all-one tensor just for debugging
vfeatures = torch.ones(100, 2000)
tfeatures = torch.ones(40, 2000)
dfeatures = torch.ones(100, 1000)
labels = torch.randint(0, 3, (100,))

In [12]:
model = dam2s.dam2s_a(num_classes=4, num_samples=100, v_dim=2000, d_dim=1000, subspace_dim=500)
problem = dam2s.cmp(500, 4)
formulation = cooper.LagrangianFormulation(problem)
primal_optimizer = torch.optim.SGD(list(model.parameters()), lr=1e-3)
dual_optimizer = cooper.optim.partial_optimizer(torch.optim.SGD, lr=1e-3)
combined = cooper.ConstrainedOptimizer(formulation, primal_optimizer, dual_optimizer)

In [13]:
for i in range(4):
    combined.zero_grad()
    lagrangian = formulation.composite_objective(
        problem.closure,
        vfeatures,
        dfeatures,
        labels,
        tfeatures,
        *model.parameters()
    )
    formulation.custom_backward(lagrangian)
    combined.step()